In [57]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/recomm.csv


In [58]:
data=pd.read_csv("/kaggle/input/recomm.csv")

In [59]:
data.head()

,userId,trailListId,name,counter
0,0,2930939,Azad khan,NaN
1,0,2931276,देवराज पांडोली,NaN
2,0,2932914,Atamram,NaN
3,0,2933829,Kisanaram nayak,1.0
4,0,2935329,Aadit,NaN


In [60]:
data.shape

(30000, 4)

In [61]:
data.isna().sum()

userId           0
trailListId      0
name             0
counter        831
dtype: int64

In [62]:
missing_pivot=data.pivot_table(values='counter',index='userId',columns='name')

In [63]:
missing_pivot.head()

name,\n I like food,☔️ Rainy Food Walk 🦌 along the Busy Street of Raja Market 🥘,"🍩 Yummilious everything! ➡ 🌼 Cafe myodo, Kalyan nagar",!! Paisa vasool !!,"""घर पर ही कैसे पाएं फेयर स्किन के साथ कुछ हफ्ते |\n""","""पुरुषों के लिए डार्क स्पॉट से छुटकारा पाने के लिए कैसे\n""","""पुरुषों के लिए सर्वश्रेष्ठ फेसवॉश तैलीय त्वचा, मुँहासे, निष्पक्षता और शुष्क त्वचा से छुटकारा पाने के लिए\n""","""भारत में 10 सर्वश्रेष्ठ हल्के शैम्पू\n""",#ContestAlert: Get Paid To Travel! APPLY NOW 🌍😎 Deadline - 31st October !,#DessertHeaven,...,"🍹 Chilling by the rooftop ➡ Ruh's Cafe, BTM 🍴","🍻 Making memories! ➡ STORIES, J P nagar 🍴",🎈Hot Air Ballooning ➡ Checked off my Bucket List ✔,🎒 Backpacking ➡ Bangalore 🔁 Munnar,"🏍️ Hot Wheels To Sula Grape Winery, Ramanagara 😎",👍🏻😍Asteria Jaipur 😍👌🏻,👣 Experiencing Goan Beach Vibes in Bengaluru! 🍹,😈Sinful Delights 😈,"🚗 CHILL IN THE HILL ➡ Kuttikkanam, Kerala 🌴",🚗 Spiti Chale hai Hum
userId,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN
24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
rate={}
row_indexes ={}
for i,row in missing_pivot.iterrows():
    rows=[x for x in range(0,len(missing_pivot.columns))]
    combine = list(zip(row.index,row.values,rows))
    rated=[(x,z)for x,y,z in combine if str(y)!='nan']
    index=[i[1]for i in rated]
    row_names=[i[0] for i in rated]
    row_indexes[i] = index
    rate[i] = row_names

In [65]:
rate

{0: ['Kisanaram nayak'],
 1: ['#ContestAlert: Get Paid To Travel! APPLY NOW 🌍😎  Deadline - 31st October !',
  '#SpottingBappa Challenge: Trails of Pandals',
  'Amusing Ganesh Pandals in Aamchi Mumbai😇',
  'Artistic expressions around me',
  'Beautiful Landscapes of Mumbai',
  'Beautiful beaches',
  'Delhi_bikerboi',
  'Diego',
  'Get Paid To Travel!',
  "Grab on presents ' Bachat Waali Diwali '\xa0",
  'Happy world music day',
  "Here's Why You Should Travel With Your Sibling! 👫👬👭 #TrellTellsATale",
  'Hidden Gems Around Bengaluru',
  'Meet the Winners of Get Paid To Travel!',
  'My Bagpacking stories',
  'My Favorite Lounges in Bengaluru',
  'Places where I go when I get bored',
  'Testing',
  'The Dazzling Diwali Project',
  'The Holi Playlist Challenge - The Weekend Challenge \U0001f91f🏻',
  'The Holi Playlist Challenge - Weekend Challenge! \U0001f91f🏻',
  'The colors of independent India',
  'Theme of the Week #1: Christmas & New Year Celebrations',
  'Trell House Moments!',
  'Wee

**user item matrix filling nan values**

In [66]:
pivot_table=data.pivot_table(values='counter',index='userId',columns='name').fillna(0)

In [67]:
pivot_table=pivot_table.apply(np.sign)

In [68]:
pivot_table.head()

name,\n I like food,☔️ Rainy Food Walk 🦌 along the Busy Street of Raja Market 🥘,"🍩 Yummilious everything! ➡ 🌼 Cafe myodo, Kalyan nagar",!! Paisa vasool !!,"""घर पर ही कैसे पाएं फेयर स्किन के साथ कुछ हफ्ते |\n""","""पुरुषों के लिए डार्क स्पॉट से छुटकारा पाने के लिए कैसे\n""","""पुरुषों के लिए सर्वश्रेष्ठ फेसवॉश तैलीय त्वचा, मुँहासे, निष्पक्षता और शुष्क त्वचा से छुटकारा पाने के लिए\n""","""भारत में 10 सर्वश्रेष्ठ हल्के शैम्पू\n""",#ContestAlert: Get Paid To Travel! APPLY NOW 🌍😎 Deadline - 31st October !,#DessertHeaven,...,"🍹 Chilling by the rooftop ➡ Ruh's Cafe, BTM 🍴","🍻 Making memories! ➡ STORIES, J P nagar 🍴",🎈Hot Air Ballooning ➡ Checked off my Bucket List ✔,🎒 Backpacking ➡ Bangalore 🔁 Munnar,"🏍️ Hot Wheels To Sula Grape Winery, Ramanagara 😎",👍🏻😍Asteria Jaipur 😍👌🏻,👣 Experiencing Goan Beach Vibes in Bengaluru! 🍹,😈Sinful Delights 😈,"🚗 CHILL IN THE HILL ➡ Kuttikkanam, Kerala 🌴",🚗 Spiti Chale hai Hum
userId,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


**trail not liked by the users**

In [69]:
notrated={}
notrated_indexes ={}
for i,row in missing_pivot.iterrows():
    rows=[x for x in range(0,len(missing_pivot.columns))]
    combine = list(zip(row.index,row.values,rows))
    idx_row=[(idx,col)for idx,val,col in combine if not val >0]
    indices=[i[1]for i in idx_row]
    row_names=[i[0] for i in idx_row]
    notrated_indexes[i] = indices
    notrated[i] = row_names

In [ ]:
notrated

**Nearest Neighbor Recommender**

In [70]:
n=5
cosine_knn= NearestNeighbors(n_neighbors=n,algorithm='brute',metric='cosine')
item_cosine_knn_fit=cosine_knn.fit(pivot_table.T.values)
item_distance,item_indices = item_cosine_knn_fit.kneighbors(pivot_table.T.values)

**Item-Based Recommender**

In [71]:
item_dic={}
for i in range (len(pivot_table.T.index)):
    item_idx = item_indices[i]
    col_names= pivot_table.T.index[item_idx].tolist()
    item_dic[pivot_table.T.index[i]]=col_names

In [72]:
item_dic

{'\n        I like food': ['Exploring Bangalore 😃',
  'Ttttttt',
  'Mommie Dressing Week 10 – Life Moves On',
  'o',
  'Moong dal dose ||పెసరట్టు ||'],
 ' ☔️ Rainy Food Walk 🦌 along the Busy Street of Raja Market 🥘': ['My Mahabaleshwar Voyage!',
  'Jehhe',
  'Tesgsgs',
  'New',
  'The jormey to become a Balcony Gardener!'],
 ' 🍩 Yummilious everything! ➡ 🌼 Cafe myodo, Kalyan nagar ': ['My treasured Sunset destinations in Udupi',
  'My treasured Sunset destinations in Union City',
  'My treasured Sunset destinations in Thiruvananthapuram',
  'My treasured Sunset destinations in Toranagallu',
  'My treasured Sunset destinations in Stuttgart'],
 '!! Paisa vasool !!': ['Amchi Mumbai',
  'All in a day plan',
  'Travel dairies ⛺',
  'Pune ka Khaana',
  'Carpe Diem'],
 '"घर पर ही कैसे पाएं फेयर स्किन के साथ कुछ हफ्ते |\n"': ['अंडर कट पुरुषों के लिए कैसे करें ',
  '"घर पर ही कैसे पाएं फेयर स्किन के साथ कुछ हफ्ते |\n"',
  'Undercut Hairstyle for Men',
  'गहरी गर्दन को कैसे हटाएँ ',
  'अपने चेहरे

In [73]:

topRecs ={}
for k,v in row_indexes.items():
    item_idx=[j for i in item_indices[v] for j in i]
    item_dist=[j for i in item_distance[v] for j in i]
    combine = list(zip(item_dist,item_idx))
    diction={i:d for d,i in combine if i not in v}
    zipped=list(zip(diction.keys(),diction.values()))
    sort = sorted(zipped, key = lambda x:x[1])
    recommendations = [(pivot_table.columns[i],d)for i,d in sort]
    topRecs[k]=recommendations
   

In [ ]:
def getrecommendations(user,number_of_recs=10):
    if user>len(pivot_table.index):
        print('out of range, there are only {} users,try again!'.format(len(pivot_table.index)))
    else:
        print("These are all the trails you have viewed  in the past: \n\n{}".format('\n'.join(rate[user])))
        print()
        print("We recommend to viewww these movies too:\n")
    for k,v in topRecs.items():
        if user ==k:
            for i in v[:number_of_recs]:
                print('{} with similarity:{:.4f}'.format(i[0],1 - i[1]))

In [74]:
getrecommendations(1)

These are all the trails you have viewed  in the past: 

#ContestAlert: Get Paid To Travel! APPLY NOW 🌍😎  Deadline - 31st October !
#SpottingBappa Challenge: Trails of Pandals
Amusing Ganesh Pandals in Aamchi Mumbai😇
Artistic expressions around me
Beautiful Landscapes of Mumbai
Beautiful beaches
Delhi_bikerboi
Diego
Get Paid To Travel!
Grab on presents ' Bachat Waali Diwali ' 
Happy world music day
Here's Why You Should Travel With Your Sibling! 👫👬👭 #TrellTellsATale
Hidden Gems Around Bengaluru
Meet the Winners of Get Paid To Travel!
My Bagpacking stories
My Favorite Lounges in Bengaluru
Places where I go when I get bored
Testing
The Dazzling Diwali Project
The Holi Playlist Challenge - The Weekend Challenge 🤟🏻
The Holi Playlist Challenge - Weekend Challenge! 🤟🏻
The colors of independent India
Theme of the Week #1: Christmas & New Year Celebrations
Trell House Moments!
Weekend Challenge #1 : The Brilliant Biryani Project 🥘
Weekend Challenge #10: The Budding Bookworm Project 
Weekend Ch

In [75]:
item_distances=1-item_distance

In [76]:
predictions =item_distance.T.dot(pivot_table.T.values) / np.array([np.abs(item_distances.T).sum(axis=1)]).T

In [77]:
ground_truth = pivot_table.T.values[item_distances.argsort()[0]]

****Evaluating the recommender's predictions****

In [78]:
def rmse(prediction,ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth= ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction,ground_truth))

In [ ]:
error_rate=rmse(predictions,ground_truth)
print("Accuracy: {:.3f}".format(100-error_rate))
print("RMSE: {:.5f}".format(error_rate))